In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-25 16:07:08--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-01-25 16:07:08 (6.07 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [6]:
# Count the number of records

print(df.count())

4864249


In [7]:
# Drop duplicates and incomplete rows
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

4864249
4863497
4863497


After removing NA's and duplicates, there are 4,863,497 records in the Amazon Review data for Toys.

In [8]:
# Create review_id_table
from pyspark.sql.functions import col
from pyspark.sql.types import DateType

review_id = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"]).withColumn('review_date', col('review_date').cast(DateType()))
review_id.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:
# Create products table

products = df.select(['product_id', 'product_title'])
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [10]:
# Create customers table

customers = df.select('customer_id').groupBy('customer_id').count().withColumnRenamed('count', 'customers_count')
customers.show(5)


+-----------+---------------+
|customer_id|customers_count|
+-----------+---------------+
|   40305706|              1|
|    3887504|              4|
|   41739251|              1|
|   28952249|             51|
|   34724923|              1|
+-----------+---------------+
only showing top 5 rows



In [11]:
# Create vine_table

vine_table = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine'])
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [18]:
# Write tables to RDS
from config import endpoint, password

# Configuration for RDS instance
mode="append"
jdbc_url = f"jdbc:postgresql://{endpoint}:5432/my_data_class_db_hh"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [19]:
# Write review table

review_id.write.jdbc(url=jdbc_url, table='Toys_review_id_table', mode=mode, properties=config)

KeyboardInterrupt: ignored

In [ ]:
# Write products table

products.write.jdbc(url=jdbc_url, table='Toys_products', mode=mode, properties=config)

In [ ]:
# Write customers table

customers.write.jdbc(url=jdbc_url, table='Toys_customers', mode=mode, properties=config)

In [ ]:
# Write vine table

vine_table.write.jdbc(url=jdbc_url, table='Toys_vine_table', mode=mode, properties=config)